### Imports used 

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
%matplotlib inline

import keras
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Flatten, Dense, Activation, Reshape

###Keras YOLO Model

In [ ]:
keras.backend.set_image_dim_ordering('th')

def make_model():
    model = Sequential()

    model.add(Convolution2D(16, 3, 3, input_shape=(3, 448, 448), border_mode='same', subsample=(1, 1)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(32, 3, 3, border_mode='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2), border_mode='valid'))
    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2), border_mode='valid'))
    model.add(Convolution2D(128, 3, 3, border_mode='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2), border_mode='valid'))
    model.add(Convolution2D(256, 3, 3, border_mode='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2), border_mode='valid'))
    model.add(Convolution2D(512, 3, 3, border_mode='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2), border_mode='valid'))
    model.add(Convolution2D(1024, 3, 3, border_mode='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Convolution2D(1024, 3, 3, border_mode='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Convolution2D(1024, 3, 3, border_mode='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Dense(4096))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(1470))

    return model

### Tensor Prediction Post Processing

In [ ]:
class Box:
    def __init__(self):
        #center coordinates (x,y) of the bounding box relative to the grid cell
        self.x, self.y = float(), float()
        #width and height relative to the entire image
        self.w, self.h = float(), float()
        #confidence, IOU between the predicted and ground truth bounding box
        self.c = float()
        #class probability per grid cell, only one is made per cell regardless of number of boxes predicted
        self.prob = float()

def overlap(x1, w1, x2, w2):
    #Subtract midpoint from width/height and maximize to find left bounding box
    l1 = x1 - w1 / 2.
    l2 = x2 - w2 / 2.
    left = max(l1, l2)
    #Same but minimize to find right bounding box
    r1 = x1 + w1 / 2.
    r2 = x2 + w2 / 2.
    right = min(r1, r2)
    #Subtract Right from Left to find overlap between bounding boxes
    overlap = right - left
    return overlap


def box_intersection(box_a, box_b):
    #find width and height of overlapping area between bbox a and b
    width = overlap(box_a.x, box_a.w, box_b.x, box_b.w)
    height = overlap(box_a.y, box_a.h, box_b.y, box_b.h)
    #if overlap is negative, there is no overlap
    if width < 0 or height < 0:
        return 0

    overlap_area = width * height
    return overlap_area


def box_union(box_a, box_b):
    #Finds the area of union between box a and b
    area_a = box_a.w * box_a.h
    area_b = box_b.w * box_b.h
    intersection = box_intersection(box_a, box_b)
    union = area_a + area_b - intersection
    return union


def box_iou(box_a, box_b):
    #Finds intersection over union for box a and b
    return box_intersection(box_a, box_b) / box_union(box_a, box_b)



def prediction_to_boundingboxes(prediction_vector, threshold=0.2, sqrt=1.8, C=20, B=2, S=7):
    # Index of car class in the VOC dataset
    car_class_number = 6

    boxes = []

    #The output of the YOLO network or the prediction vector is a 1470(S*S*(B*5+C)) vector that contains the information for predicting the bounding boxes
    SS = S * S  # number of grid cells
    prob_size = SS * C  # class probabilities 7*7*20=980
    conf_size = SS * B  # confidences for each grid cell 7*7*2=98

    #first part of vector where probability predictions are
    probabilities = prediction_vector[0:prob_size]
    #second part of vector where confidence are found
    confidence_scores = prediction_vector[prob_size: (prob_size + conf_size)]
    #last part of vector where bounding box predictions are
    cords = prediction_vector[(prob_size + conf_size):]

    # Reshape the arrays so that its easier to loop over them
    probabilities = probabilities.reshape((SS, C))
    confs = confidence_scores.reshape((SS, B))
    cords = cords.reshape((SS, B, 4))

    #Loop to find x,y,w,h,and c predictions from output vector
    for grid in range(SS):
        for b in range(B):
            bx = Box()

            bx.c = confs[grid, b]

            #Converting position vectors from absolute value to relative
            bx.x = (cords[grid, b, 0] + grid % S) / S
            bx.y = (cords[grid, b, 1] + grid // S) / S

            bx.w = cords[grid, b, 2] ** sqrt
            bx.h = cords[grid, b, 3] ** sqrt

            # multiply confidence scores with class probabilities to get class sepcific confidence scores
            p = probabilities[grid, :] * bx.c

            # Check if the confidence score for class 'car' is greater than the threshold
            if p[car_class_number] >= threshold:
                bx.prob = p[car_class_number]
                boxes.append(bx)

    # sort confidence score of each box in starting from largest to smallest
    boxes.sort(key=lambda b: b.prob, reverse=True)

    #remove overlapping boxes
    for i in range(len(boxes)):
        boxi = boxes[i]
        #discards predicted box if probability is zero
        if boxi.prob == 0:
            continue

        for j in range(i + 1, len(boxes)):
            boxj = boxes[j]

            # If boxes have 40% iou then the boxes with the highest probability is kept
            if box_iou(boxi, boxj) >= 0.4:
                boxes[j].prob = 0

    boxes = [b for b in boxes if b.prob > 0]

    return boxes


def draw_boxes(boxes, img, crop_dim=((500,1280),(300,650))):
    drawn_img = img.copy()
    #crop image so we dont draw boxes where there would be no cars, such as in the sky or in the lane past the barrier
    [xmin, xmax] = crop_dim[0]
    [ymin, ymax] = crop_dim[1]

    height, width, _ = drawn_img.shape
    for box in boxes:
        w = xmax - xmin
        h = ymax - ymin

        left = int((box.x - box.w / 2.) * w) + xmin
        right = int((box.x + box.w / 2.) * w) + xmin
        top = int((box.y - box.h / 2.) * h) + ymin
        bot = int((box.y + box.h / 2.) * h) + ymin

        if left < 0:
            left = 0
        if right > width - 1:
            right = width - 1
        if top < 0:
            top = 0
        if bot > height - 1:
            bot = height - 1

        thick = 5
        color = (0, 0, 255)  #blue

        cv2.rectangle(drawn_img, (left, top), (right, bot), color, thick)

    return drawn_img

### Making model and showing model summary

In [ ]:
from helper_functions import load_weights

model = make_model()
load_weights(model,'yolo-tiny.weights')
model.summary()

### Detection Pipeline

In [ ]:
from helper_functions import preprocess

def detection_pipeline(img):
    pre_processed = preprocess(img)
    #expand dimensions because input expects [1,3,448,448]
    input = np.expand_dims(pre_processed, axis=0)
    prediction = model.predict(input)
    #Post process vector to extract bounding boxes
    bboxes = prediction_to_boundingboxes(prediction[0],threshold=0.20)
    #draw boxes
    final_img = draw_boxes(bboxes,img)
    return final_img

In [ ]:
import matplotlib.gridspec as gs

def plot_all_imgs(input,output):
    plt.figure(figsize=(15,10))
    gs1 = gs.GridSpec(nrows=1,ncols=2)
    
    ax1 = plt.subplot(gs1[0,0])
    ax1.set_title('Original')
    plt.imshow(input)
    
    ax2 = plt.subplot(gs1[0,1])
    ax2.set_title('Bounding Boxes')
    plt.imshow(output)
    
    plt.show()

### Running pipeline on test images

In [ ]:
test_images = glob.glob('test_images/test*.jpg')
for img in test_images:
    input = mpimg.imread(img)
    output = detection_pipeline(input)
    plot_all_imgs(input,output)

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

###Pipeline on project video

In [ ]:
output_video = 'video_output.mp4'
output_clip = VideoFileClip('project_video.mp4')
project_clip = output_clip.fl_image(detection_pipeline)  # NOTE: this function expects color images!!
project_clip.write_videofile(output_video, audio=False)

## References
- [You Only Look Once (YOLO) paper](https://arxiv.org/abs/1506.02640)
- [Darknet](https://github.com/pjreddie/darknet)
- [Darknet to Keras (YAD2K)](https://github.com/allanzelener/YAD2K)
- [Xslittlegrass Implemintation](https://github.com/xslittlegrass/CarND-Vehicle-Detection)
- [Subodh Malgonde Implemination](https://github.com/subodh-malgonde/vehicle-detection)

